# Start with loading the glove embedding

In [1]:
import numpy as np 
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from torch import nn
import math
from functools import partial
from pathlib import Path
from tqdm import tqdm
#import rich
from typing import List, Tuple, Optional, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import transformers
#import tokenizers
#import datasets
#import zipfile
#from huggingface_hub import hf_hub_download
device = 'cpu'

In [2]:
target_file = "glove.6B.50d.txt"
vocabulary = []
vectors = []
emb_dic = {}
with open(target_file, "r", encoding="utf8") as f:
    for l in tqdm(f.readlines() ):
        word, *vector = l.split()
        vocabulary.append(word)
        vector_t = torch.tensor([float(v) for v in vector])
        vectors.append(vector_t)
        emb_dic[word] = vector_t
vectors = torch.stack(vectors)

100%|██████████| 400001/400001 [00:08<00:00, 48825.03it/s]


In [3]:
# somtehing from week 4
dim_emb = 50
class NewsEncoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.activation_fn = nn.ReLU()
        self.mul_head_att = nn.MultiheadAttention(dim_emb,1)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_k = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_v = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        
        self.ff1 = nn.Linear(in_features=dim_emb, out_features=128)
        self.ff2 = nn.Linear(in_features=128, out_features=num_classes)

        #self.l_1 = nn.Linear(in_features=int(features_cat_size),
        #                  out_features=l_1_hidden_size)
        

    def forward(self, x): # x is a string of words - title
        words = x.split(" ")
        vectors = []
        for word in words:
            print(word)
            vectors.append(torch.Tensor(emb_dic[word.lower()]))
        #features_final = self.l_1(x)
        e_s = torch.stack(vectors)
        q = self.lin_q(e_s)
        k = self.lin_k(e_s)
        v = self.lin_v(e_s)
        attn_out, attn_weight = self.mul_head_att(q,k,v)
        #print(attn_out)
        cls_tok_emb = attn_out[0,:]
        print(cls_tok_emb)
        
        output = self.ff1(cls_tok_emb)
        output = self.activation_fn(output)
        output = self.ff2(output)
        
        return output
    


In [4]:
model = NewsEncoder(5) # TODO - I need to change this with a number that will correspond to 

In [5]:
model

NewsEncoder(
  (activation_fn): ReLU()
  (mul_head_att): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
  )
  (lin_q): Linear(in_features=50, out_features=50, bias=True)
  (lin_k): Linear(in_features=50, out_features=50, bias=True)
  (lin_v): Linear(in_features=50, out_features=50, bias=True)
  (ff1): Linear(in_features=50, out_features=128, bias=True)
  (ff2): Linear(in_features=128, out_features=5, bias=True)
)

In [6]:
test1 = "Breaking news about this"
test2 = "This has happened again"
#test3 = "Natascha var ikke den foerste"
#test4 = "Kun Star Wars tjente mere"

In [7]:
attn_output = model.forward(test2)
shape = attn_output.shape
attn_output.reshape(shape)

This
has
happened
again
tensor([ 6.4050e-02,  5.3003e-02,  3.0495e-01,  2.7242e-02, -7.4303e-02,
        -1.2375e-01,  4.5723e-02, -1.6454e-02,  1.5557e-01,  2.3183e-01,
        -3.6420e-04,  2.2021e-01, -2.4660e-02,  1.1702e-01,  1.3349e-01,
         3.0678e-01,  1.6261e-01,  6.5852e-02, -1.4067e-01, -5.7109e-02,
         6.3530e-02,  4.5563e-02,  3.8250e-02, -1.1695e-01, -9.1109e-02,
        -7.5147e-02, -3.4102e-02, -1.7600e-02,  3.6804e-01,  3.5737e-02,
        -7.7090e-02, -3.1313e-01, -2.3903e-02, -2.5665e-01,  3.9103e-01,
        -2.1776e-01, -1.9104e-02, -2.2428e-01,  7.5744e-03,  9.9495e-02,
         1.4469e-01, -2.1187e-01, -1.9310e-01, -2.9819e-01, -6.9273e-02,
         2.2607e-01, -1.0778e-01,  1.5534e-01, -4.4466e-03, -2.1049e-01],
       grad_fn=<SliceBackward0>)


tensor([ 0.0263,  0.0323, -0.0106, -0.1667,  0.0241], grad_fn=<ViewBackward0>)

# brand new encoder

In [31]:
class MyEmbeddingLayer(nn.Module):
    def __init__(self,emb_dim, emb_dic):
        super().__init__()
        self.emb_dim = emb_dim
        self.emb_dic = emb_dic
        
    def forward(self,text):
        input_shape = text.shape
        titles = text.flatten()
        output = []
        for title in titles:
            words = title.split(" ")
            for word in words:
                output.append(torch.Tensor(self.emb_dic[word.lower()]))
            
        output = torch.stack(output)
        output = output.reshape(input_shape + (-1,self.emb_dim))
        return output
            

In [32]:
test = MyEmbeddingLayer(50, emb_dic)
input = np.array([test1,test2])
print(input.shape)
#input = torch.from_numpy(input)
test.forward(input)

(2,)


tensor([[[-7.5307e-01, -1.4580e-02, -7.5133e-02,  1.0766e-01, -1.1661e-02,
           4.1505e-01,  7.2856e-02,  4.9378e-01,  1.4302e-01,  2.7014e-01,
          -2.9928e-01, -4.9729e-01, -1.1358e+00,  5.7973e-01,  5.5898e-01,
          -3.1587e-01, -5.7323e-02, -7.2313e-01, -4.0444e-01, -2.3284e-01,
           1.8744e-01,  3.2356e-01, -1.6800e-02, -3.3481e-01,  2.2256e-01,
          -1.3342e+00,  4.9411e-01,  8.8150e-02,  8.0475e-01, -1.7204e-01,
           2.7073e+00,  1.2706e-01,  9.3040e-02,  3.3197e-01, -7.0747e-01,
           2.3589e-01,  3.0587e-01,  1.6879e-01, -1.8252e-01, -1.2134e-01,
          -2.3045e-01, -1.5842e-01,  5.3029e-02,  4.9780e-01,  1.1690e-01,
           4.4732e-02,  2.3904e-01, -4.5940e-01, -4.5749e-01, -6.2330e-01],
         [-2.0825e-01,  4.7786e-01,  5.2196e-01,  1.0587e+00, -1.0045e-01,
          -1.1269e+00, -1.2581e+00, -1.1041e-01, -7.4125e-02, -7.7976e-01,
          -3.7942e-01, -2.4860e-01, -3.9224e-01,  4.2972e-01,  9.8060e-01,
           1.2668e-01, -

### version 2

In [33]:
# somtehing from week 4
dim_emb = 50
class MyNewsEncoder(nn.Module):
    def __init__(self, out_dimension, head_count=1):
        super().__init__()
        self.out_dimension = out_dimension
        self.activation_fn = nn.Tanh()
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.softmax_dim1 = nn.Softmax(dim=1)
        self.softmax = nn.Softmax()
        self.v_net = nn.Linear(dim_emb, dim_emb)
        #--------
        self.mul_head_att = nn.MultiheadAttention(dim_emb,1)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_k = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_v = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        
        self.ff1 = nn.Linear(in_features=dim_emb, out_features=128)
        self.ff2 = nn.Linear(in_features=128, out_features=out_dimension)

    def forward(self, x): # x is a string of words - title
        #words = x.split(" ")
        #vectors = []
        #for word in words:
        #    print(word)
        #    vectors.append(torch.Tensor(emb_dic[word.lower()]))
        #features_final = self.l_1(x)
        e_s = self.embedding(x)
        q = self.lin_q(e_s)
        k = self.lin_k(e_s)
        v = self.lin_v(e_s)
        a_k = self.get_A_Matrix(e_s, q)
        h_ks = self.get_h_head_vectors(v,a_k,e_s)
        h = self.combine_heads(h_ks)
        
        a_w = q.T @ self.activation_fn(self.v_net(h))
        a_w = self.softmax(a_w)
        
        
        attn_out, attn_weight = self.mul_head_att(q,k,v)
        #print(attn_out)
        cls_tok_emb = attn_out[0,:]
        print(cls_tok_emb)
        
        output = self.ff1(cls_tok_emb)
        output = self.activation_fn(output)
        output = self.ff2(output)
        
        return output
    
    def get_A_Matrix(e,Q):
        matr_mult = e.T @ Q.T @ e
        a = softmax_dim1(matr_mult)
        return a
    
    def get_h_head_vectors(V, a, e):
        inner_dot = a @ e # TODO check if the dimensions really make dot product
        h = V @ inner
        return h
    
    def combine_heads(h_ks):
        return h_ks # TODO change this when I will have multi head

        
    


### version 3

In [11]:
# somtehing from week 4
dim_emb = 50
class MyNewsEncoder(nn.Module):
    def __init__(self, out_dimension, head_count=1):
        super().__init__()
        self.out_dimension = out_dimension
        self.activation_fn = nn.Tanh()
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.softmax_dim1 = nn.Softmax(dim=1) # TODO do I go for the correct dimension?
        self.softmax = nn.Softmax(dim=1)
        #--------
        self.lin_qk = nn.Linear(dim_emb, dim_emb, bias=False)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=1)
        self.lin_vk = nn.Linear(in_features=dim_emb,out_features=dim_emb, bias=False)
        self.lin_vw = nn.Linear(in_features=dim_emb,out_features=dim_emb)

    def forward(self, x): # x is a string of words - title
        e_s = self.embedding(x)
        
        # starting single head
        qe = self.lin_qk(e_s)
        et_qt = qe @ e_s.transpose(-2,-1)
        ak = self.softmax_dim1(et_qt)
        print((ak @ e_s).shape)
        hk = self.lin_vk(ak @ e_s)
        h = hk # TODO make it multi head
        print('h shape', h.shape)
        # end of single head
        
        aw = self.lin_q(self.activation_fn(self.lin_vw(h)))
        print('AW',aw.shape)
        aw = self.softmax(aw)
        print(aw.shape, h.shape)
        r = aw.transpose(-2,-1) @ h # just confirmed by calculation that this seems correct
        return r.reshape(-1,50)
    


### version 4

In [35]:
dim_emb = 50

class SelfAttHead(nn.Module):
    def __init__(self, dim_emb, head_out):
        super().__init__()
        self.lin_qk = nn.Linear(dim_emb, dim_emb, bias=False)
        self.softmax_dim1 = nn.Softmax(dim=1) # TODO do I go for the correct dimension?
        self.lin_vk = nn.Linear(in_features=dim_emb,out_features=head_out, bias=False)
        
    def forward(self,x):
        qe = self.lin_qk(x)
        et_qt = qe @ x.transpose(-2,-1)
        ak = self.softmax_dim1(et_qt)
        hk = self.lin_vk(ak @ x)
        return hk
    
    
class MyNewsEncoder(nn.Module):
    def __init__(self, embedding_dimension, head_count=1):
        super().__init__()
        assert embedding_dimension % head_count == 0, "embeding must be divisible by heads"
        self.head_out = embedding_dimension // head_count
        self.activation_fn = nn.Tanh()
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.softmax = nn.Softmax(dim=1)
        #--------
        self.lin_vw = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=1, bias=False)
        
        self.selfAtt = nn.ModuleList([SelfAttHead(dim_emb, self.head_out) for i in range(head_count)])

    def forward(self, x): # x is a string of words - title
        e_s = self.embedding(x)
        
        hk = []
        for head in self.selfAtt:
            att = head(e_s)
            hk.append(att)
        h = torch.cat(hk, 2)
        print(h.shape)
        
        aw = self.lin_q(self.activation_fn(self.lin_vw(h)))
        aw = self.softmax(aw)
        r = aw.transpose(-2,-1) @ h # just confirmed by calculation that this seems correct
        return r.reshape(-1,50)
    


### checking DK

In [36]:
model = MyNewsEncoder(dim_emb, head_count=10)
model(input)

torch.Size([2, 4, 50])


tensor([[ 0.0717,  0.0292,  0.1425,  0.1842, -0.5162,  0.0396,  0.5044, -0.0452,
          0.0755, -0.3381, -0.8201,  0.5778,  0.4399, -0.1889,  0.5134,  0.2353,
          0.4658, -0.1613, -0.2020,  0.2898,  0.1932, -0.3018, -0.3265, -0.1332,
         -0.3535, -0.3792, -0.2867, -0.3376, -0.2533,  0.3144,  0.0669, -0.3916,
         -0.4766, -0.0251,  0.4266, -0.0782, -0.4342, -0.3937, -0.2354,  0.1222,
         -0.2173,  0.1128,  0.3991,  0.0846,  0.6094, -0.1019,  0.1692,  0.0449,
         -0.2952,  0.0126],
        [-0.0104,  0.1340,  0.2028,  0.0213, -0.2697, -0.0581,  0.1869, -0.0831,
         -0.0571, -0.3982, -0.8542,  0.3998,  0.4479, -0.0935,  0.4947,  0.0428,
          0.4897,  0.1213, -0.4090,  0.1667,  0.2823, -0.4390, -0.3521, -0.0259,
         -0.5940, -0.3555, -0.4330, -0.0341, -0.1675,  0.2652, -0.1117, -0.1740,
         -0.6428, -0.1229,  0.5833,  0.1539, -0.1460, -0.4751, -0.0909, -0.0585,
         -0.2951,  0.2540,  0.3135,  0.1013,  0.6803, -0.0254, -0.0519,  0.2482,


In [42]:
loss = nn.L1Loss()
inp = torch.randn(3,5, requires_grad=True)
target = torch.randn(3,5)
output = loss(inp, target)
output.backward()

In [45]:
loss = nn.L1Loss()
output_tmp = model(input)
target = torch.randn(2,50)
fake_loss = loss(output_tmp, target)

fake_loss.backward()

torch.Size([2, 4, 50])


### version 5

In [58]:
word_count = 27693 + 1
MAX_WORDS = 30
class MyEmbeddingLayer(nn.Module):
    def __init__(self,emb_dim, emb_dic):
        super().__init__()
        self.emb_dim = emb_dim
        self.emb_dic = emb_dic
        self.emb_torch = nn.Embedding(word_count, emb_dim)
        self.dummy_emb = dummy_dictionary_embedding
        
    def forward(self,text):
        input_shape = text.shape
        titles = text.flatten()
        output = []
        for title in titles:
            words = title.split(" ")
            for word in words:
                #print('inside: ', word.lower(), ' -> ', self.dummy_emb[word.lower()])
                output.append(torch.IntTensor([self.dummy_emb[word.lower()]]))
            for i in range(30 - len(words)):
                output.append(torch.IntTensor([word_count-1]))
        
        #print(output[0].shape, output[1].shape)
        output = torch.stack(output)
        output = self.emb_torch(output)
        output = output.reshape(input_shape + (-1,self.emb_dim))
        return output

In [59]:
dim_emb = 50

class SelfAttHead(nn.Module):
    def __init__(self, dim_emb, head_out):
        super().__init__()
        self.lin_qk = nn.Linear(dim_emb, dim_emb, bias=False)
        self.softmax_dim1 = nn.Softmax(dim=1) # TODO do I go for the correct dimension?
        self.lin_vk = nn.Linear(in_features=dim_emb,out_features=head_out, bias=False)
        
    def forward(self,x):
        qe = self.lin_qk(x)
        et_qt = qe @ x.transpose(-2,-1)
        ak = self.softmax_dim1(et_qt)
        hk = self.lin_vk(ak @ x)
        return hk
    
class MultiHeadSelfAttHead(nn.Module):
    def __init__(self,embedding_dimension, head_count):
        super().__init__()
        self.head_out = embedding_dimension // head_count # TODO this will be later more specific
        self.selfAtt = nn.ModuleList([SelfAttHead(dim_emb, self.head_out) for _ in range(head_count)])
        
    def forward(self, e_s):
        hk = []
        for head in self.selfAtt:
            att = head(e_s)
            hk.append(att)
        h = torch.cat(hk, 2)
        return h

class AdditiveWordAttention(nn.Module):
    def __init__(self, embedding_dimension, additive_vector_dim):
        super().__init__()
        self.activation_fn = nn.Tanh()
        self.lin_vw = nn.Linear(in_features=embedding_dimension, out_features=additive_vector_dim)
        self.lin_q = nn.Linear(in_features=additive_vector_dim, out_features=1, bias=False)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, h):
        aw = self.lin_q(self.activation_fn(self.lin_vw(h)))
        aw = self.softmax(aw)
        r = aw.transpose(-2,-1) @ h
        return r
    
class MyNewsEncoder(nn.Module):
    def __init__(self, embedding_dimension, head_count=1):
        super().__init__()
        assert embedding_dimension % head_count == 0, "embeding must be divisible by heads"
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.mult_head_att = MultiHeadSelfAttHead(embedding_dimension, head_count)
        self.add_word_att = AdditiveWordAttention(embedding_dimension, embedding_dimension) # TODO later change the vector dim to 200

    def forward(self, x): # x is a string of words - title
        e_s = self.embedding(x)
        
        h = self.mult_head_att(e_s)
        
        r = self.add_word_att(h)
        return r.squeeze(dim=-2)

### Vesrion 6

In [ ]:
class UserEncoder(nn.Module):
    def __init__(self, emb_dimension, user_head_count=1, news_head_count=1):
        super().__init__()
        self.news_encoder = MyNewsEncoder(emb_dimension, news_head_count)
        self.multi_head_att = MultiHeadSelfAttHead(emb_dimension, head_count)
        self.add_news_att = AdditiveWordAttention(emb_dimension,emb_dimension)
    
    def forward(self,x):
        
        r = self.news_encoder(x)
        
        h = self.multi_head_att(r)
        
        u = self.add_news_att(h)
        
        return u.squeeze(dim=-2)

### checking

In [60]:
dk_input = np.array(["Natascha var ikke den første", "Kun Star Wars tjente mere", "Luderne flytter på landet"])

In [61]:
loss = nn.L1Loss()
target = torch.randn(3,50)
model = MyNewsEncoder(dim_emb, head_count=10)
output_tmp = model(dk_input)
tmp_test_loss = loss(output_tmp, target)
tmp_test_loss.backward()

In [62]:
simplemat = torch.rand(2,3)
simplemat, simplemat.sum(dim=1)

(tensor([[0.5886, 0.1133, 0.6954],
         [0.7556, 0.0775, 0.5810]]),
 tensor([1.3973, 1.4141]))

# loading data

### laoding input

In [16]:
import pyarrow.parquet as pq
from transformers import AutoTokenizer, AutoModel

/run/media/lukisko/Windows-SSD/DTU/SEMESTER 2/deep-learning/project/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [221]:
table_behavior = pq.read_table('ebnerd_small/train/behaviors.parquet')
table_history = pq.read_table('ebnerd_small/train/history.parquet')
table_articles = pq.read_table('ebnerd_small/articles.parquet')
df_behavior = table_behavior.to_pandas()
df_history = table_history.to_pandas()
df_articles = table_articles.to_pandas()

In [222]:
df_articles.head(10)

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3001353,Natascha var ikke den første,"Politiet frygter nu, at Nataschas bortfører ha...",2023-06-29 06:20:33,False,Sagen om den østriske Natascha og hendes bortf...,2006-08-31 08:06:45,[3150850],article_default,https://ekstrabladet.dk/krimi/article3001353.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9955,Negative
1,3003065,Kun Star Wars tjente mere,Biografgængerne strømmer ind for at se 'Da Vin...,2023-06-29 06:20:35,False,Vatikanet har opfordret til at boykotte filmen...,2006-05-21 16:57:00,[3006712],article_default,https://ekstrabladet.dk/underholdning/filmogtv...,...,[],"[Underholdning, Film og tv, Økonomi]",414,"[433, 434]",underholdning,NaN,NaN,NaN,0.8460,Positive
2,3012771,Morten Bruun fyret i SønderjyskE,FODBOLD: Morten Bruun fyret med øjeblikkelig v...,2023-06-29 06:20:39,False,Kemien mellem spillerne i Superligaklubben Søn...,2006-05-01 14:28:40,[3177953],article_default,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,...,[],"[Erhverv, Kendt, Sport, Fodbold, Ansættelsesfo...",142,"[196, 199]",sport,NaN,NaN,NaN,0.8241,Negative
3,3023463,Luderne flytter på landet,I landets tyndest befolkede områder skyder bor...,2023-06-29 06:20:43,False,Det frække erhverv rykker på landet. I den tyn...,2007-03-24 08:27:59,[3184029],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Livsstil, Erotik]",118,[133],nyheder,NaN,NaN,NaN,0.7053,Neutral
4,3032577,Cybersex: Hvornår er man utro?,En flirtende sms til den flotte fyr i regnskab...,2023-06-29 06:20:46,False,"De fleste af os mener, at et tungekys er utros...",2007-01-18 10:30:37,[3030463],article_default,https://ekstrabladet.dk/sex_og_samliv/article3...,...,[],"[Livsstil, Partnerskab]",565,[],sex_og_samliv,NaN,NaN,NaN,0.9307,Neutral
5,3033563,Kniven for struben-vært får selv kniven,I aftenens udgave af 'Med kniven for struben' ...,2023-06-29 06:20:47,False,Når man ser fjerde program i TV3s nye programs...,2007-03-27 10:22:08,"[3005524, 3005525]",article_default,https://ekstrabladet.dk/underholdning/filmogtv...,...,[],"[Livsstil, Underholdning, Film og tv, Kultur, ...",414,"[433, 436]",underholdning,NaN,NaN,NaN,0.9371,Neutral
6,3034608,Willy Strube har begået selvmord,"Den tidligere SiD-chef tog sit eget liv, efter...",2023-06-29 06:20:49,False,Den tidligere formand for Industrigruppen i Si...,2001-10-19 12:30:00,[3204848],article_default,https://ekstrabladet.dk/nyheder/politik/articl...,...,"[PER, PER, PER]","[Kriminalitet, Erhverv, Privat virksomhed, Ans...",118,[130],nyheder,NaN,NaN,NaN,0.9971,Negative
7,3034849,Venner for livet,VK-REGERINGEN,2023-06-29 06:20:50,False,VK-REGERINGEN\nhåndplukkede Bjørn Lomborg som ...,2003-01-09 06:00:00,None,article_default,https://ekstrabladet.dk/incoming/article303484...,...,[],"[Kendt, Politik, National politik]",2,[],incoming,NaN,NaN,NaN,0.8454,Neutral
8,3035648,Dronning af escort-branchen,"Trine Michelsen hjælper københavnsk bagmand, s...",2023-06-29 06:20:52,False,En af escortbranchens største bagmandvirksomhe...,2003-06-17 07:10:00,[3082573],article_default,https://ekstrabladet.dk/krimi/article3035648.ece,...,[],"[Erhverv, Livsstil, Transportmiddel, Erotik]",140,[],krimi,NaN,NaN,NaN,0.8814,Neutral
9,3036444,Mia kendte sandsynligvis sin morder,"Hun var ikke den type, der søgte kontakt til f...",2023-06-29 06:20:54,False,Den 12-årige Mia Teglgaard Sprotte fra Benløse...,2003-07-13 19:50:00,None,article_default,https://ekstrabladet.dk/krimi/article3036444.ece,...,"[PER, PER, PER, PER]","[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9752,Negative


In [19]:
titles = df_articles['title'].values
titles

array(['Natascha var ikke den første', 'Kun Star Wars tjente mere',
       'Morten Bruun fyret i SønderjyskE', ...,
       'Dansk skuespiller: - Jeg nægtede, at jeg var syg',
       'Så slemt er det: 14.000 huse er oversvømmet',
       'Aktion mod svindlere: Seks personer anholdt'], dtype=object)

In [20]:
split_words = lambda tit : tit.split(' ')
all_words = list(map(split_words, titles))
words = []
for title in all_words:
    for word in title:
        words.append(word.lower())

In [21]:
all_words = np.unique(np.sort(np.array(words)))

In [22]:
print(all_words.shape)
dummy_dictionary_embedding = {}
for i in range(len(all_words)):
    dummy_dictionary_embedding[all_words[i]] = i

(27693,)


In [23]:
X_all = df_articles['title'].values
Y_all = df_articles['category_str'].values

In [24]:
categories = np.unique(df_articles['category_str'].values)
cat_map = {}
for i in range(len(categories)):
    cat_map[categories[i]] = i
print(cat_map)

{'auto': 0, 'bibliotek': 1, 'biler': 2, 'dagsorden': 3, 'ferie': 4, 'forbrug': 5, 'haandvaerkeren': 6, 'horoskoper': 7, 'incoming': 8, 'krimi': 9, 'musik': 10, 'nationen': 11, 'nyheder': 12, 'om_ekstra_bladet': 13, 'opinionen': 14, 'penge': 15, 'play': 16, 'plus': 17, 'podcast': 18, 'services': 19, 'sex_og_samliv': 20, 'side9': 21, 'sport': 22, 'underholdning': 23, 'video': 24, 'vin': 25}


In [205]:
y_temp_embd = nn.Embedding(26,dim_emb)

In [26]:
cat_to_index = lambda a: cat_map[a]

In [27]:
Y_all_index = torch.IntTensor(list(map(cat_to_index, Y_all)))

In [206]:
Y_all_vec = y_temp_embd(Y_all_index).detach()

In [207]:
X_all.shape, Y_all_vec.shape

((20738,), torch.Size([20738, 50]))

## training

In [63]:
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import torch.optim as optim

In [177]:
class CustomDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, index):
        #print(self.X_train[index], self.y_train[index])
        return [self.X_train[index], self.y_train[index]]

In [208]:
len(X_all)

20738

In [209]:
tmp_dk_input = np.array(['Natascha var ikke den første','Kun Star Wars tjente mere','Luderne flytter på landet'])
target = torch.randn(3,50)

In [210]:
type(X_all), type(tmp_dk_input)
X_all.size, tmp_dk_input.size
X_all[0], tmp_dk_input[0]

('Natascha var ikke den første', np.str_('Natascha var ikke den første'))

In [211]:
type(Y_all_vec), type(target)
Y_all_vec.size(), target.size()
Y_all_vec[0], target[0]

(tensor([ 0.8113, -0.5601,  0.0688,  0.0433,  0.6322,  1.7011, -0.6115,  1.5072,
         -0.5583, -1.2399,  0.4653, -2.5312, -0.8267,  0.1064,  0.8190,  0.8980,
         -0.7079,  0.8811, -1.6164, -0.6474, -0.4017, -0.1428, -0.3780, -0.4416,
          0.3583, -0.9101,  0.6233,  0.0609, -0.9289,  0.2133,  1.6290,  0.4116,
          2.2968,  0.1557, -0.3821,  1.9859,  0.3232, -0.6153, -0.8077, -0.0276,
          0.1847,  0.5487,  2.3812,  0.1613,  1.5597,  1.5464,  0.1718, -0.2927,
          0.4616,  1.3618]),
 tensor([-0.8797, -0.2395, -0.9081, -2.0901, -0.1674,  0.0502,  1.0764,  0.5941,
          0.4634,  1.4409, -0.6228, -1.3523,  1.4419,  1.3898,  1.1106, -0.0901,
         -0.7134,  0.1914,  0.2988, -1.5603, -2.6037, -0.6166,  0.1844, -0.7520,
          2.0305, -0.0928,  2.7589,  0.3643, -2.0165,  1.2678,  1.1598,  0.3436,
          1.8406, -1.3440, -0.1751, -1.1208, -1.1207,  0.4806,  0.8751, -0.7550,
          0.4259, -0.4042,  0.2590, -1.1447, -1.2505,  0.7114,  1.8315,  1.0575,

In [217]:
model = MyNewsEncoder(50)
train_dataset = CustomDataset(X_all, Y_all_vec)#CustomDataset(tmp_dk_input,target)#CustomDataset(X_all, Y_all_vec)
print(X_all[0])
loss_fn = nn.L1Loss() #nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Natascha var ikke den første


In [218]:
tmp_dk_input = ('Natascha var ikke den første','Kun Star Wars tjente mere','Luderne flytter på landet')

In [220]:
batch_size = 64
num_epochs = 20
validation_every_steps = 40

# Make data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
##validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

step = 0
model.train()

train_accuracies = []
train_loss = []
validation_accuracies = []
validation_loss = []
        
for epoch in range(num_epochs):
    
    train_accuracies_batches = []
    train_loss_batches = []
    
    for inputs, targets in train_loader:#[(tmp_dk_input, target)]:#train_loader:#[(dk_input, target)]:#train_loader:
        #print(targets)
        # Forward pass.
        #print('broken',inputs)
        #print('working',target)
        output = model(np.array(inputs))#model(np.array(tuple(dk_input)))#model(np.array(inputs))
        
        # Compute loss.
        loss = loss_fn(output, targets)
        train_loss_batches.append(loss.detach().numpy())
        # Clean up gradients from the model.
        optimizer.zero_grad()
        
        # Compute gradients based on the loss from the current batch (backpropagation).
        loss.backward()
        
        # Take one optimizer step using the gradients computed in the previous step.
        optimizer.step()
        
        step += 1
        
        # Compute accuracy.
        predictions = output.max(1)[1]
        targ_ind = targets.max(1)[1]
        #print('out:', output)
        #print('predictions:', predictions)
        #print('targets:', targ_ind)
        train_accuracies_batches.append(accuracy_score(targ_ind, predictions))
        
        
        if step % validation_every_steps == 0:
            
            # Append average training accuracy to list.
            train_accuracies.append(np.mean(train_accuracies_batches))
            train_loss.append(np.mean(train_loss_batches))
            
            train_accuracies_batches = []
            train_loss_batches = []
        
            # Compute accuracies on validation set.
            # validation_accuracies_batches = []
            # with torch.no_grad():
            #     model.eval()
            #     for inputs, targets in validation_loader:
            #         output = model(inputs)
            #         loss = loss_fn(output, targets.float())

            #         predictions = output.max(1)[1]
            #         targ_ind = targets.max(1)[1]
                    
            #         # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
            #         validation_accuracies_batches.append(accuracy_score(targ_ind, predictions) * len(inputs))

            #     model.train()
                
            # # Append average validation accuracy to list.
            # validation_accuracies.append(np.sum(validation_accuracies_batches) / len(validation_dataset))
     
            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}, loss: {train_loss[-1]}")
            #print(f"             validation accuracy: {validation_accuracies[-1]}")

print("Finished training.")

Step 40      training accuracy: 0.0, loss: 0.7235375642776489
Step 80      training accuracy: 0.0, loss: 0.7236472368240356
Step 120     training accuracy: 0.0, loss: 0.723263144493103
Step 160     training accuracy: 0.0, loss: 0.7244390249252319
Step 200     training accuracy: 0.0, loss: 0.7234910130500793
Step 240     training accuracy: 0.0, loss: 0.7228919267654419
Step 280     training accuracy: 0.0, loss: 0.7216987609863281
Step 320     training accuracy: 0.0, loss: 0.7200679183006287
Step 360     training accuracy: 0.0, loss: 0.7220596671104431
Step 400     training accuracy: 0.0, loss: 0.7213143110275269
Step 440     training accuracy: 0.0, loss: 0.720506489276886
Step 480     training accuracy: 0.0, loss: 0.7168058753013611
Step 520     training accuracy: 0.0, loss: 0.7228422164916992
Step 560     training accuracy: 0.0, loss: 0.7214546203613281
Step 600     training accuracy: 0.0, loss: 0.7274776697158813
Step 640     training accuracy: 0.0, loss: 0.7192741632461548
Step 680  

In [131]:
testoing_this = next(iter(train_loader))

In [134]:
testing_this_input, testing_target = testoing_this

In [138]:
len(testing_this_input[0])

64

In [129]:
model = MyNewsEncoder(50)
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 2
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, target in train_loader:
        optimizer.zero_grad()  # Clean up gradients from the model
        
        output = model(np.array(inputs))  # Forward pass
        loss = loss_fn(output, target)  # Compute the loss
        print(loss.shape)
        
        total_loss += loss.item()  # Accumulate the loss

    total_loss.backward()  # Backpropagate the accumulated loss
    optimizer.step()  # Update model weights

    print(f'Epoch {epoch + 1}, Total Loss: {total_loss}')


torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size

AttributeError: 'float' object has no attribute 'backward'

In [133]:
epochs = 5

for e in range(epochs):
    train_loss = 0.0
    data = X_all 
    labels = Y_all_vec
    for i in range(1):        
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = loss_fn(target,labels)
        # Calculate gradients 
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()
    
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)}')

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

# translating

In [21]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from torch import nn
import math
from functools import partial
from pathlib import Path
from tqdm import tqdm
import rich
from typing import List, Tuple, Optional, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import transformers
import tokenizers
import datasets
import zipfile
from huggingface_hub import hf_hub_download
import pyarrow.parquet as pq


In [22]:
task_name = f"translation_da_to_en"
model_name = f"Helsinki-NLP/opus-mt-da-en"
translator  = transformers.pipeline(task_name, model=model_name)

/run/media/lukisko/Windows-SSD/DTU/SEMESTER 2/deep-learning/project/env/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [23]:
translator(['Natascha var ikke den første','Kun Star Wars tjente mere'])

[{'translation_text': "Natascha wasn't the first."},
 {'translation_text': 'Only Star Wars made more'}]

In [24]:
table_articles = pq.read_table('ebnerd_small/articles.parquet')
df_articles = table_articles.to_pandas()

In [29]:
emb_dic['t']

tensor([-0.3792,  0.6185,  0.9593,  0.9040,  0.3681,  0.0230,  0.1680, -1.5309,
        -0.0605, -0.2500,  0.1503,  0.3197, -0.6891, -0.7863, -0.0158,  0.5053,
        -0.8473, -0.1235,  0.0785, -0.9602, -0.5431, -0.3352,  0.3893,  0.1946,
        -1.1688, -0.8661, -0.3918,  0.2418,  0.3286, -0.7875,  2.4884,  0.7102,
        -0.5311,  0.8959, -0.2305, -0.8202,  0.3442, -0.9687, -0.1514, -0.4491,
         0.8951, -0.0166, -0.2749,  0.2795,  0.7794, -0.3194,  0.1676, -0.6252,
         0.0533,  0.6202])

In [30]:
df_articles.shape

(20738, 21)

# links
https://sanjayasubedi.com.np/deeplearning/multihead-attention-from-scratch/

https://github.com/ebanalyse/ebnerd-benchmark/blob/67a352b73306bd03aeff824f2451867314a26414/examples/00_quick_start/nrms_ebnerd.ipynb#L238

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)#, weight_decay=0.3)

NameError: name 'optim' is not defined

In [ ]:
# Test the forward pass with dummy data
out = model([])
print("Output shape:", out.size())
print(f"Output logits:\n{out.detach().cpu().numpy()}")
print(f"Output probabilities:\n{out.softmax(1).detach().cpu().numpy()}")